# Preprocessing Pipeline

plus, bonus Naive Bayes Model at the end

In [1]:
import pandas as pd
import re
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pkg_resources
#!pip install symspellpy
from symspellpy import SymSpell, Verbosity
import nltk
from nltk.stem import WordNetLemmatizer

from datetime import datetime
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S %z %Z")

In [2]:
reviews = pd.read_csv("scraped_comments.csv", parse_dates=['date'], date_parser=dateparse).sample(n=50000, random_state=1)

In [3]:
reviews.drop_duplicates(subset="comment_id", keep="first", inplace=True)

# TODO: move dropped rows into another CSV file and 

reviews.dropna(subset=["comment"], inplace=True)
# drop rows containing only "No Comments" (default value assigned by RMP to a review that didn't enter a comment)
reviews = reviews[reviews["comment"] != "No Comments"]
# replace all comments with less than 5 words with a NaN
reviews["comment"] = reviews["comment"].apply(lambda x: x if len(x.split()) > 5 else None)
# drop rows containing NaN comment
reviews.dropna(subset=["comment"], inplace=True)

reviews.reset_index(drop=True, inplace=True)

Replacing emoticons

In [4]:
EMOTICONS = {
    u":‑)":"emo_pos",
    u":-))":"emo_pos",
    u":-)))":"emo_pos",
    u":)":"emo_pos",
    u":))":"emo_pos",
    u":)))":"emo_pos",
    u":-]":"emo_pos",
    u":]":"emo_pos",
    u":-3":"emo_pos",
    u":3":"emo_pos",
    u":->":"emo_pos",
    u":>":"emo_pos",
    u"8-)":"emo_pos",
    u":-}":"emo_pos",
    u":}":"emo_pos",
    u":-)":"emo_pos",
    u":c)":"emo_pos",
    u":^)":"emo_pos",
    u"=]":"emo_pos",
    u"=)":"emo_pos",
    u":‑D":"emo_pos",
    u":D":"emo_pos",
    u"8‑D":"emo_pos",
    u"8D":"emo_pos",
    u"X‑D":"emo_pos",
    u"XD":"emo_pos",
    u"=D":"emo_pos",
    u"=3":"emo_pos",
    u"B^D":"emo_pos",
    u":-))":"emo_pos",
    u":-(":"emo_neg",
    u":‑(":"emo_neg",
    u":(":"emo_neg",
    u":‑c":"emo_neg",
    u":c":"emo_neg",
    u":‑<":"emo_neg",
    u":<":"emo_neg",
    u":‑[":"emo_neg",
    u":[":"emo_neg",
    u":-||":"emo_neg",
    u">:[":"emo_neg",
    u":{":"emo_neg",
    u":@":"emo_neg",
    u">:(":"emo_neg",
    u":'‑(":"emo_neg",
    u":'(":"emo_neg",
    u":'‑)":"emo_pos",
    u":')":"emo_pos",
    u"D‑':":"emo_neg",
    u"D:<":"emo_neg",
    u"D:":"emo_neg",
    u"D8":"emo_neg",
    u"D;":"emo_neg",
    u"D=":"emo_neg",
    u"DX":"emo_neg",
    u";‑)":"emo_pos",
    u";)":"emo_pos",
    u"*-)":"emo_pos",
    u"*)":"emo_pos",
    u";‑]":"emo_pos",
    u";]":"emo_pos",
    u";^)":"emo_pos",
    u":‑,":"emo_pos",
    u";D":"emo_pos",
    u":‑P":"emo_pos",
    u":P":"emo_pos",
    u"X‑P":"emo_pos",
    u"XP":"emo_pos",
    u":‑Þ":"emo_pos",
    u":Þ":"emo_pos",
    u"=p":"emo_pos",
    u":‑/":"emo_neg",
    u":/":"emo_neg",
    u":-[.]":"emo_neg",
    u">:[(\)]":"emo_neg",
    u">:/":"emo_neg",
    u":[(\)]":"emo_neg",
    u"=/":"emo_neg",
    u"=[(\)]":"emo_neg",
    u":L":"emo_neg",
    u"=L":"emo_neg",
    u":‑|":"emo_neg",
    u":|":"emo_neg",
    u"O:‑)":"emo_pos",
    u"O:)":"emo_pos",
    u"0:‑3":"emo_pos",
    u"0:3":"emo_pos",
    u"0:‑)":"emo_pos",
    u"0:)":"emo_pos",
    u":‑b":"emo_pos",
    u"(>_<)":"emo_neg",
    u"(>_<)>":"emo_neg",
    u"^_^":"emo_pos",
    u"(^_^)/":"emo_pos",
    u"(^O^)／":"emo_pos",
    u"(^o^)／":"emo_pos",
    u"('_')":"emo_neg",
    u"(/_;)":"emo_neg",
    u"(T_T) (;_;)":"emo_neg",
    u"(;_;":"emo_neg",
    u"(;_:)":"emo_neg",
    u"(;O;)":"emo_neg",
    u"(:_;)":"emo_neg",
    u"(ToT)":"emo_neg",
    u";_;":"emo_neg",
    u";-;":"emo_neg",
    u";n;":"emo_neg",
    u"Q.Q":"emo_neg",
    u"T.T":"emo_neg",
    u"Q_Q":"emo_neg",
    u"(-.-)":"emo_pos",
    u"(-_-)":"emo_pos",
    u"(；一_一)":"emo_pos",
    u"(=_=)":"emo_neg",
    u"^m^":"emo_pog",
    u">^_^<":"emo_pos",
    u"<^!^>":"emo_pos",
    u"^/^":"emo_pos",
    u"（*^_^*）" :"emo_pos",
    u"(^<^) (^.^)":"emo_pos",
    u"(^^)":"emo_pos",
    u"(^.^)":"emo_pos",
    u"(^_^.)":"emo_pos",
    u"(^_^)":"emo_pos",
    u"(^^)":"emo_pos",
    u"(^J^)":"emo_pos",
    u"(*^.^*)":"emo_pos",
    u"(^—^）":"emo_pos",
    u"(#^.^#)":"emo_pos",
    u"(*^0^*)":"emo_pos",
    u"(*^^)v":"emo_pos",
    u"(^_^)v":"emo_pos",
    u'(-"-)':"emo_neg",
    u"(ーー;)":"emo_neg",
    u"(＾ｖ＾)":"emo_pos",
    u"(＾ｕ＾)":"emo_pos",
    u"(^)o(^)":"emo_pos",
    u"(^O^)":"emo_pos",
    u"(^o^)":"emo_pos",
    u")^o^(":"emo_pos",
    u":O o_O":"emo_neg",
    u"o_0":"emo_neg",
    u"o.O":"emo_neg",
    u"(o.o)":"emo_neg",
    u"(*￣m￣)": "emo_neg",
}

def replace_emoticons(text):
    return EMOTICONS.get(text, text)

Expanding contractions

In [5]:
contraction_mapping = {
    "ain't": "is not", 
    "aren't": "are not",
    "can't": "cannot",
    "'cause": "because", 
    "could've": "could have", 
    "couldn't": "could not", 
    "didn't": "did not",  
    "doesn't": "does not", 
    "don't": "do not", 
    "hadn't": "had not", 
    "hasn't": "has not", 
    "haven't": "have not", 
    "he'd": "he would",
    "he'll": "he will", 
    "he's": "he is", 
    "how'd": "how did", 
    "how'd'y": "how do you", 
    "how'll": "how will", 
    "how's": "how is",  
    "I'd": "I would", 
    "I'd've": "I would have", 
    "I'll": "I will", 
    "I'll've": "I will have",
    "I'm": "I am", 
    "I've": "I have", 
    "i'd": "i would", 
    "i'd've": "i would have", 
    "i'll": "i will",  
    "i'll've": "i will have",
    "i'm": "i am", 
    "i've": "i have", 
    "isn't": "is not", 
    "it'd": "it would", 
    "it'd've": "it would have", 
    "it'll": "it will", 
    "it'll've": "it will have",
    "it's": "it is", 
    "let's": "let us", 
    "ma'am": "madam", 
    "mayn't": "may not", 
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have", 
    "must've": "must have", 
    "mustn't": "must not",
    "mustn't've": "must not have"}

def expand_contraction(text): # Before expanding contraction, might want to clean of symbols that are not '
  return contraction_mapping.get(text, text)

Spell Checking

In [6]:
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt"
)
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# lookup suggestions for single-word input strings
#input_term = "memebers"  # misspelling of "members"

# Verbosity.TOP gets the best suggestion
#suggestion = sym_spell.lookup(input_term, Verbosity.TOP, max_edit_distance=2)
#print(suggestion[0], len(suggestion))

True

In [7]:
# amend list of stop words to keep whatever it is we want by removing words from list that we want to keep

# TODO: is the list of stopwords on git complete and accurate or does someone want to read through all 325 stopwords spacy gives and determine which ones to keep?
stopwords = STOP_WORDS
stopwords.remove("but")
stopwords.remove("not")
stopwords.remove("nor")
stopwords.remove("never")

In [10]:
comments_proper = []
#spellchecked_comments = []
lemm = WordNetLemmatizer()

for i in range(reviews.shape[0]):
    review = reviews["comment"][i]
    review = re.sub('&([a-zA-z]+|#\d+);', "", review)           # remove HTML codes
    review = re.sub('&#63;?', '', review)                       # HTML code for question mark evades erasure on occasion, handle here
    review = re.sub(r'\s*https?://\S+(\s+|$)', ' ', review)                                     # remove links
    review = re.sub("^(\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}$", ' ', review)         # remove phone numbers
    review = re.sub("[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+", " ", review)              # remove email addresses

    review_split = review.split()
    for i, text in enumerate(review_split):
        text = replace_emoticons(text)
        text = text.lower()
        text = expand_contraction(text)
        review_split[i] = text
    review = " ".join(review_split)
    
    #review = re.sub('\d{4,}', ' ', review)                      # keep numbers between 1 and 3 digits
    review = re.sub('[^a-zA-Z\s_]+', ' ', review)              # remove non-alphabetic characters, spaces, and any remaining digits

    review = re.sub(r'(.)\1\1+', '\g<1>', review)               # replace any three characters in a row with one
    
    review = re.sub('\s+', ' ', review)
    review = review.lower()                                     # lowercase review for uniformity

    review = review.split()
    review = [lemm.lemmatize(word) for word in review if word not in stopwords]
    review = " ".join(review)

    comments_proper.append(review)
    #spellchecked_comments.append(' '.join(sym_spell.lookup(word, Verbosity.TOP, max_edit_distance=2, include_unknown=True)[0].term for word in review.split()))

In [ ]:
"""
nlp = spacy.load("en_core_web_md", disable=["parser", "ner", "textcat"])

def lemmatize_pipe(doc):
    lemma_list = [str(tok.lemma_).lower() for tok in doc if tok.is_alpha and tok.text.lower() not in stopwords]
    return lemma_list

def preprocess_pipe(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=20):
        preproc_pipe.append(lemmatize_pipe(doc))
    return preproc_pipe
"""

In [11]:
no_stopwords = comments_proper

In [12]:
reviews = reviews.loc[:, ["firstName", "lastName", "comment", "clarityRating"]]
reviews["cleanedComment"] = pd.Series([" ".join(comm) for comm in no_stopwords])
#reviews["cleanedCommentChecked"] = pd.Series([" ".join(comm) for comm in no_stopwords_checked])
reviews.head()

,firstName,lastName,comment,clarityRating,cleanedComment
0,Keith,Halladay,Keith is the best. Not afraid to give his hone...,5,k e i t h b e s t n o t a f r a i d h ...
1,Charles,Lankau,Lankau was great for 4000 and 5110. He makes c...,4,l a n k a u g r e a t m a k e c l a s s ...
2,James,McGivern,I hated every second of my life that I had to ...,2,h a t e d s e c o n d l i f e s i t c ...
3,Howard G.,Tucker,Crazy Man. No grading system. You dont know yo...,5,c r a z y m a n g r a d i n g s y s t e ...
4,Paul,Cohen,This teacher is the worst I've ever taken in m...,1,t e a c h e r w o r s t t a k e n e n t ...


In [13]:
#reviews["sentiment"] = reviews["clarityRating"].apply(lambda x: 1 if x > 3 else 0 if x == 3 else -1)       # accuracy: 0.7518830305715551 for 50000 comments
reviews["sentiment"] = reviews["clarityRating"].apply(lambda x: 1 if x > 2.5 else 0)                        # accuracy: 0.8635356668143553 for 50000 comments
reviews.head()

,firstName,lastName,comment,clarityRating,cleanedComment,sentiment
0,Keith,Halladay,Keith is the best. Not afraid to give his hone...,5,k e i t h b e s t n o t a f r a i d h ...,1
1,Charles,Lankau,Lankau was great for 4000 and 5110. He makes c...,4,l a n k a u g r e a t m a k e c l a s s ...,1
2,James,McGivern,I hated every second of my life that I had to ...,2,h a t e d s e c o n d l i f e s i t c ...,0
3,Howard G.,Tucker,Crazy Man. No grading system. You dont know yo...,5,c r a z y m a n g r a d i n g s y s t e ...,1
4,Paul,Cohen,This teacher is the worst I've ever taken in m...,1,t e a c h e r w o r s t t a k e n e n t ...,0


##### Writing cleaned comments to a new csv file so we (hopefully) don't have to run this everytime

In [14]:
reviews.to_csv("comments_preproc.csv", index=False)

## Naive Bayes model

In [25]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500, ngram_range=(1, 1))
X = cv.fit_transform(no_stopwords).toarray()

In [26]:
y = pd.get_dummies(reviews.loc[:, ["sentiment"]])     # isolate sentiments
y = y.loc[:, "sentiment"]

In [34]:
from sklearn.model_selection import train_test_split
#X_valid = X[0:20000]
#y_valid = y[0:20000]

#X = X[20000:]
#y = y[20000:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [35]:
from sklearn.naive_bayes import MultinomialNB
#from sklearn.naive_bayes import BernoulliNB
model = MultinomialNB().fit(X_train, y_train)
#model = BernoulliNB().fit(X_train, y_train)
y_pred = model.predict(X_test)

#y_pred = model.predict(X_valid)

In [36]:
# Test accuracy of model
from sklearn.metrics import confusion_matrix, accuracy_score

conf_m = confusion_matrix(y_test, y_pred)

acc_score = accuracy_score(y_test, y_pred)

print("Accuracy Score: " + str(acc_score * 100))
print(conf_m)

Accuracy Score: 85.87727071333629
[[1735  606]
 [ 669 6018]]


*Matrix read like:*

* __Top left:__ 0s correctly identifed as 0s (*true negative*)
* __Top right:__ 0s incorrectly predicted as 1s (*false positive*)
* __Bottom left:__ 1s incorrectly predicted as 0s (*false negative*)
* __Bottom right:__ 1s correctly identifed as 1s (*true positive*)

Can extend to 3x3 case, though more complicated, see https://towardsdatascience.com/understanding-the-confusion-matrix-from-scikit-learn-c51d88929c79 for a better explanation than I can muster :)

In [37]:
import nltk

all_words = " ".join(comments_proper)
all_words = all_words.split()
bgs = nltk.bigrams(all_words)
bigram_feature_vector = []
for item in nltk.bigrams(all_words):
    bigram_feature_vector.append(" ".join(item))
bigram_feature_vector

['keith best',
 'best not',
 'not afraid',
 'afraid honest',
 'honest opinion',
 'opinion grading',
 'grading paper',
 'paper choosing',
 'choosing class',
 'class discussion',
 'discussion super',
 'super laid',
 'laid funny',
 'funny write',
 'write paper',
 'paper work',
 'work good',
 'good lankau',
 'lankau great',
 'great make',
 'make class',
 'class fun',
 'fun hilarious',
 'hilarious prepared',
 'prepared study',
 'study help',
 'help but',
 'but not',
 'not going',
 'going easy',
 'easy definitely',
 'definitely work',
 'work grade',
 'grade but',
 'but make',
 'make worth',
 'worth listening',
 'listening lecture',
 'lecture hated',
 'hated second',
 'second life',
 'life sit',
 'sit class',
 'class intelligent',
 'intelligent man',
 'man but',
 'but idea',
 'idea teach',
 'teach crazy',
 'crazy man',
 'man grading',
 'grading system',
 'system dont',
 'dont know',
 'know grade',
 'grade tell',
 'tell student',
 'student access',
 'access available',
 'available expects',
 '

## Calculate Precision and Recall of Model

See https://en.m.wikipedia.org/wiki/Precision_and_recall

In [38]:
# Precision wrt 0
prec_0 = conf_m[0][0] / float(conf_m[1][0] + conf_m[0][0])

# Recall wrt 0
rec_0 = conf_m[0][0] / float(conf_m[0][1] + conf_m[0][0])

print("Precision={0}; Recall={1}".format(prec_0,rec_0))

Precision=0.7217138103161398; Recall=0.7411362665527552


In [39]:
# Precision wrt 1
prec_1 = conf_m[1][1] / float(conf_m[0][1] + conf_m[1][1])

# Recall wrt 1
rec_1 = conf_m[1][1] / float(conf_m[1][0] + conf_m[1][1])

print("Precision={0}; Recall={1}".format(prec_1,rec_1))

Precision=0.9085144927536232; Recall=0.8999551368326604


In [40]:
# More refined way to calculate precision and recall, should've done my research first lol
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_test, y_pred, average="weighted")
recall = recall_score(y_test, y_pred, average="weighted")

print("Precision={0}; Recall={1}".format(precision, recall))

Precision=0.8600762564237441; Recall=0.8587727071333628
